In [1]:
import pandas as pd
import numpy as np
import os
import requests
from io import BytesIO
import json
import yadisk
from datetime import datetime, date, timedelta
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import config
from yandex_disk_func import *
from parse_functions import *
from google_connector import *
from date_time_func import *

# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'
# забираем токен для подключения к гугл
service_key = config.service
google_sheet_link = 'https://docs.google.com/spreadsheets/d/1eakG0BJF1eSyi0hyGUpdw166TH_C9h_2zjnyrsMeejk/edit?usp=sharing' #'https://docs.google.com/spreadsheets/d/11hF_txYEbKrQImRQ0x3Fia3PyJvDAqtpGI25idV7oa4/edit?usp=sharing'
gmail = config.gmail
sheet_name = 'prog'

# обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета
public_key = config.public_key 

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
main_dict = {}

In [ ]:
get_data_from_ya_folder(yandex_folders, main_dict, flag='prog')                     


In [ ]:
df = pd.concat(main_dict, ignore_index=True)
df = df.fillna('')

df['views'] = df['views'].astype('float').round(0)
int_columns = ['reach', 'impressions', 'clicks', 'views']
df[int_columns] = df[int_columns].apply(lambda x: x.astype('int'))

In [ ]:
float_columns = ['unit price', 'budget_without_nds', 'budget_nds', 'vtr, %']
df[float_columns] = df[float_columns].apply(lambda x: x.astype('float')).round(2)

In [ ]:
df.columns

In [ ]:
# download_columns_list = ['supplier', 'report_name', 'sheet_name', 'brand', 'geo', 'soc_dem', 'ad copy format', 'rotation type', 'frequency', 'reach',
# 'impressions', 'clicks', 'budget_without_nds', 'budget_nds', 'views', 'vtr, %']

In [ ]:
download_columns_list = df.columns

In [ ]:
final_df = df[download_columns_list]

In [ ]:
final_df

In [ ]:
# создаем подключение к Гуглу
client = create_connection(service_key)

In [ ]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [ ]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, final_df)

In [3]:
# создаем кастомный календарь для медиаплана
sheet_name = 'weekYear'
calendar_df = get_mediaplan_calendar(cur_year='2024', start_day_num=1)

start: 2024-01-01, end: 2024-12-31


In [5]:
# создаем подключение к Гуглу
client = create_connection(service_key)

Connection established successfully...


In [6]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [7]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, calendar_df)

DataFrame exported successfully...


In [ ]:
df

In [ ]:
test = df[df['report_name']=='рсхб_игроник_ segmento _ рсхб_up_2024']

In [ ]:
test = df[df['sheet_name']=='mediaplan 1000к']

In [ ]:
test

In [ ]:
test.iloc[1]

In [ ]:
test[['reach', 'impressions', 'clicks', 'budget_without_nds']].sum()

In [ ]:
df

In [ ]:
main_dict['beeline_plan_игроник_ингосстрах_кис_лето_banners_020724']

In [ ]:
main_dict['plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1)'].iloc[0]

In [ ]:
main_dict['plan_ингосстрах_ beeline_хореографы_banners_240924 (3)']

In [ ]:
main_dict['beeline_plan_igronik_бриф по тендерам_101024']